In [21]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Data Preparation

In [22]:
DATA_PATH = "../../data/"
#TARGET_CITY = str(input("Which City You want to draw network?"))

In [3]:
#data = pd.read_excel(f"{DATA_PATH}df_filter_dummy_pohang.xlsx")

In [38]:
# data = pd.read_pickle(f"{DATA_PATH}df_filter_dummy_{TARGET_CITY}.pkl")

In [23]:
data = pd.read_pickle("../../data/pkl/merged_filtered_0630.pkl")

In [27]:
data = data[data['city'] == '태백시']

In [28]:
keyword_set = data['filter_keyword']

In [41]:
# keyword_set

2917491    {조성, 주민, 관광, 지역, 주민들, 건설}
2917492                {실업, 지자체, 지역}
2917493                     {주민, 지역}
2917494                       {환경오염}
2917495                         {지역}
                     ...            
2952569                         {도시}
2952570                    {발생, 주민들}
2952571                         {주민}
2952572                         {주민}
2952573                         {사업}
Name: filter_keyword, Length: 35083, dtype: object

In [29]:
key_set = []
for item in keyword_set:
    item = str(item)
    item = item.replace("{", "")
    item = item.replace("}", "")
    item = item.replace("'", "")
    splited = item.split(",")
    temp = []
    for tok in splited:
        temp.append(tok.replace(" ", ""))
    key_set.append(temp)

### Apriori algorithm

In [30]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [31]:
te = TransactionEncoder()
te_result = te.fit(key_set).transform(key_set)

In [32]:
key_df = pd.DataFrame(te_result, columns=te.columns_)

In [33]:
itemset = apriori(key_df, min_support=0.001, use_colnames=True)

In [47]:
for idx, item in enumerate(itemset.itemsets):
    if len(item) == 5:
        idx
        break
itemset_ngram = itemset[:idx]

In [12]:
from mlxtend.frequent_patterns import association_rules
apriori_result = association_rules(itemset, metric="lift", min_threshold=1)

In [13]:
apriori_result

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(건설),(감소),0.116447,0.038140,0.004821,0.041404,1.085583,0.000380,1.003405
1,(감소),(건설),0.038140,0.116447,0.004821,0.126413,1.085583,0.000380,1.011408
2,(감소),(고령화),0.038140,0.003512,0.001038,0.027208,7.746623,0.000904,1.024359
3,(고령화),(감소),0.003512,0.038140,0.001038,0.295455,7.746623,0.000904,1.365221
4,(감소),(농업),0.038140,0.021121,0.001389,0.036417,1.724174,0.000583,1.015874
...,...,...,...,...,...,...,...,...,...
103265,(사업),"(관광, 인프라, 지역, 유치, 산업, 도시, 조성)",0.279431,0.001133,0.001006,0.003599,3.175467,0.000689,1.002475
103266,(유치),"(관광, 인프라, 지역, 사업, 산업, 도시, 조성)",0.102829,0.001756,0.001006,0.009781,5.569708,0.000825,1.008104
103267,(산업),"(관광, 인프라, 지역, 사업, 유치, 도시, 조성)",0.146956,0.001229,0.001006,0.006844,5.567547,0.000825,1.005654
103268,(도시),"(관광, 인프라, 지역, 사업, 유치, 산업, 조성)",0.160701,0.001054,0.001006,0.006259,5.939879,0.000836,1.005238


In [34]:
#apriori_result.to_excel("./result/Pohang0713_apriori_result.xlsx")

In [3]:
# apriori_result = pd.read_excel("./result/Pohang0712_apriori_result.xlsx")

In [35]:
network_data = apriori_result[['antecedents', 'consequents', 'lift']]

In [36]:
network_data

,antecedents,consequents,lift
0,(건설),(감소),1.085583
1,(감소),(건설),1.085583
2,(감소),(고령화),7.746623
3,(고령화),(감소),7.746623
4,(감소),(농업),1.724174
...,...,...,...
103265,(사업),"(관광, 인프라, 지역, 유치, 산업, 도시, 조성)",3.175467
103266,(유치),"(관광, 인프라, 지역, 사업, 산업, 도시, 조성)",5.569708
103267,(산업),"(관광, 인프라, 지역, 사업, 유치, 도시, 조성)",5.567547
103268,(도시),"(관광, 인프라, 지역, 사업, 유치, 산업, 조성)",5.939879


### Visualize network

In [37]:
import matplotlib.pyplot as plt
import networkx as nx

In [1]:
# import matplotlib
# import matplotlib.font_manager

# [f.name for f in matplotlib.font_manager.fontManager.ttflist if 'Nanum' in f.name]

Duplicate key in file PosixPath('/home/mglee/.conda/envs/svmglee/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'), line 257 ('font.family:  NanumGothic')


['NanumGothic',
 'NanumSquare',
 'NanumGothicCoding',
 'Nanum Pen Script',
 'NanumMyeongjo',
 'NanumSquare',
 'NanumSquareRound',
 'NanumGothic',
 'NanumGothic',
 'Nanum Brush Script',
 'NanumMyeongjo Eco',
 'NanumBarunGothic',
 'NanumMyeongjo',
 'NanumBarunpen',
 'NanumMyeongjo',
 'NanumGothic Eco',
 'NanumBarunGothic',
 'NanumSquareRound',
 'NanumSquare',
 'NanumGothic',
 'NanumGothic Eco',
 'NanumGothic Eco',
 'NanumSquareRound',
 'NanumBarunGothic',
 'NanumGothicCoding',
 'NanumBarunGothic',
 'NanumMyeongjo Eco',
 'NanumSquareRound',
 'NanumMyeongjo Eco',
 'NanumBarunpen',
 'NanumSquare',
 'NanumGothic',
 'NanumGothic Eco',
 'NanumGothic',
 'NanumMyeongjo',
 'NanumMyeongjo',
 'NanumBarunpen',
 'NanumGothic Eco',
 'NanumMyeongjo Eco',
 'NanumMyeongjo',
 'NanumMyeongjo Eco',
 'NanumMyeongjo',
 'Nanum Pen Script',
 'NanumSquareRound',
 'NanumMyeongjo',
 'NanumBarunGothic',
 'NanumGothic',
 'Nanum Brush Script',
 'NanumGothic',
 'NanumGothicCoding',
 'NanumSquare',
 'NanumSquareRound',

In [23]:
#np.mean(network_data['lift'])

In [38]:
node_lst = []
for node in network_data['antecedents']:
    node_lst.append(', '.join(list(node)))

In [39]:
graph = nx.DiGraph()
#graph.add_nodes_from(node_lst)
for i in range(len(network_data)):
    a, b, c = network_data.iloc[i, :]
    a = ', '.join(list(a))
    b = ', '.join(list(b))
    # if a == '인구':
    #     if c > 5:
    graph.add_edge(a, b, weight = round(c,3))
    # else:
    #     pass

In [40]:
import matplotlib.font_manager as fm
import matplotlib

font_path = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
fontprop = fm.FontProperties(fname=font_path, size=18)
font_name = fm.FontProperties(fname="/usr/share/fonts/truetype/nanum/NanumGothic.ttf").get_name()

matplotlib.rcParams['font.family'] ='NanumGothic'
matplotlib.rcParams['axes.unicode_minus'] =False

In [41]:
edges,weights = zip(*nx.get_edge_attributes(graph,'weight').items())

In [42]:
pd.DataFrame(weights).quantile([0.99])

,0
0.99,17.543


In [43]:
graph.nodes('인구')

NodeDataView({'건설': None, '감소': None, '고령화': None, '농업': None, '도시': None, '도심': None, '발생': None, '사업': None, '산업': None, '유치': None, '인구': None, '인프라': None, '제조업': None, '조성': None, '지역': None, '지역경제': None, '지역사회': None, '지자체': None, '피해': None, '혁신': None, '활성화': None, '관광': None, '개최': None, '국제': None, '의료': None, '지역발전': None, '지역주민': None, '체험': None, '공업': None, '기반시설': None, '재개발': None, '재건축': None, '주민': None, '주민들': None, '지방세': None, '환경오염': None, '경제활동': None, '공공시설': None, '생활환경': None, '쇠퇴': None, '지역기업': None, '사업체': None, '실업': None, '환경문제': None, '서비스업': None, '감소, 사업': None, '개최, 사업': None, '감소, 개최': None, '건설, 도시': None, '도시, 감소': None, '건설, 감소': None, '건설, 발생': None, '건설, 사업': None, '건설, 산업': None, '산업, 감소': None, '건설, 유치': None, '감소, 유치': None, '건설, 인구': None, '감소, 인구': None, '건설, 조성': None, '감소, 조성': None, '건설, 주민': None, '감소, 주민': None, '지역, 건설': None, '지역, 감소': None, '건설, 피해': None, '감소, 피해': None, '도시, 관광': None, '감소, 관광': None, '관광, 사업': None, '산업, 관광': No

In [44]:
n_size = dict(graph.degree)
pos = nx.kamada_kawai_layout(graph)

nx.draw(graph, nodelist=n_size.keys(),
        with_labels = True, font_family = font_name, font_size = 2,
        alpha = 0.7,
        edgelist=edges, edge_color=weights, width=1, edge_cmap=plt.cm.binary)

nx.write_graphml_lxml(graph, "Taeback.graphml")

#plt.title("temp의 keyword network")
#plt.show(block=False)
#plt.savefig("temp.png", format="PNG", dpi = 1000)